In [1]:
import spacy
import os
import xapian
import re
import io
import shutil
import datetime
from tqdm import tqdm as tqdm

In [2]:
dbpath = 'data/xapIndex_SenID_Text'
datapath = 'data/wiki-pages-text/'

# Shard Paths

In [3]:
filesInDataPath = sorted([datapath + fileName for fileName in os.listdir(datapath)], reverse=True)
filesInDataPath = sorted(filesInDataPath)

In [4]:
filesInDataPath

['data/wiki-pages-text/wiki-001.txt',
 'data/wiki-pages-text/wiki-002.txt',
 'data/wiki-pages-text/wiki-003.txt',
 'data/wiki-pages-text/wiki-004.txt',
 'data/wiki-pages-text/wiki-005.txt',
 'data/wiki-pages-text/wiki-006.txt',
 'data/wiki-pages-text/wiki-007.txt',
 'data/wiki-pages-text/wiki-008.txt',
 'data/wiki-pages-text/wiki-009.txt',
 'data/wiki-pages-text/wiki-010.txt',
 'data/wiki-pages-text/wiki-011.txt',
 'data/wiki-pages-text/wiki-012.txt',
 'data/wiki-pages-text/wiki-013.txt',
 'data/wiki-pages-text/wiki-014.txt',
 'data/wiki-pages-text/wiki-015.txt',
 'data/wiki-pages-text/wiki-016.txt',
 'data/wiki-pages-text/wiki-017.txt',
 'data/wiki-pages-text/wiki-018.txt',
 'data/wiki-pages-text/wiki-019.txt',
 'data/wiki-pages-text/wiki-020.txt',
 'data/wiki-pages-text/wiki-021.txt',
 'data/wiki-pages-text/wiki-022.txt',
 'data/wiki-pages-text/wiki-023.txt',
 'data/wiki-pages-text/wiki-024.txt',
 'data/wiki-pages-text/wiki-025.txt',
 'data/wiki-pages-text/wiki-026.txt',
 'data/wiki-

Xapian build Index

In [5]:
x_db = xapian.WritableDatabase(dbpath, xapian.DB_CREATE_OR_OPEN)

Set termgenerator for indexing

In [6]:
index = xapian.TermGenerator()

Set Stemmer

In [7]:
index.set_stemmer(xapian.Stem('en'))

Build Index

In [ ]:
indexCounter = 0
print('Time started at', datetime.datetime.now().time())
with tqdm(total=len(filesInDataPath)) as pbar:
    for shardFile in filesInDataPath:
        with open(shardFile, 'r') as openedFile:
            print('Now processing:', shardFile)
            for line in openedFile:
                indexCounter += 1
                # Processing line
                docID, sentenceID, text = line.split(' ', 2)
                try:
                    sentenceID = int(sentenceID)
                except Exception:
                    pass
                xapianDoc = xapian.Document()
                xapianDoc.set_data(text + ',,,' + docID +',,,' + str(sentenceID))
                index.set_document(xapianDoc)
                index.index_text(text)
                index.increase_termpos()
                x_db.replace_document(indexCounter, xapianDoc)
        x_db.commit()
        pbar.update(1)
x_db.close()

  0%|          | 0/109 [00:00<?, ?it/s]

Time started at 14:48:27.741458
Now processing: data/wiki-pages-text/wiki-001.txt


  1%|          | 1/109 [00:28<51:34, 28.66s/it]

Now processing: data/wiki-pages-text/wiki-002.txt


  2%|▏         | 2/109 [00:58<51:59, 29.15s/it]

...

Now processing: data/wiki-pages-text/wiki-077.txt


 71%|███████   | 77/109 [2:44:49<1:45:32, 197.90s/it]

Now processing: data/wiki-pages-text/wiki-078.txt


 72%|███████▏  | 78/109 [2:48:16<1:43:41, 200.68s/it]

Now processing: data/wiki-pages-text/wiki-079.txt


Check DB statistics

In [ ]:
!xapian-delve $dbpath

Search DB

In [ ]:
TOP_RESULTS_LIMIT = 15

In [ ]:
def search(dbpath, querystring, offset=0, pagesize=10):
    
    database = xapian.Database(dbpath)
    enquire = xapian.Enquire(database)
    query_string = querystring

    qp = xapian.QueryParser()
    stemmer = xapian.Stem("english")
    qp.set_stemmer(stemmer)
    qp.set_database(database)
    
    qp.set_stemming_strategy(xapian.QueryParser.STEM_SOME)
    query = qp.parse_query(query_string)
    #print "Parsed query is: %s" % str(query)

    # Find the top results for the query.
    enquire.set_query(query)
    matches = enquire.get_mset(0, TOP_RESULTS_LIMIT)

    # Display the results.
    #print(%i results found." % matches.get_matches_estimated()
    #print "Results 1-%i:" % matches.size()

    for m in matches:
        print('RANK:', m.rank + 1)
        print('PERCENTAGE MATCH:', m.percent)
        print('DOC ID:', m.docid)
        print('DOC TXT:', m.document.get_data())

In [ ]:
query = "Xanax"
matches = search(dbpath, query)